In [1]:
!pip install pyyaml h5py

In [2]:
import os
import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.18.0


In [3]:
!python --version

Python 3.11.11


In [4]:
(train_images,train_labels), (test_images,test_labels) = tf.keras.datasets.mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0
train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0

In [6]:
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10,activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model

model = create_model()
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 512)                 │         401,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           5,130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 407,050 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# prompt: użyj callbacks w punktach kontrolnych - utwórz katalog training_1, w nim plik cp.ckpt.weights.h5, zdefiniuj model Checkpoint tylko z zapisem wag, zdefiniuj i uruchom trening sieci z użyciem callbacks

import os

# Define the checkpoint directory and file path
checkpoint_path = 'training_1/cp.ckpt.weights.h5'
checkpoint_dir = os.path.dirname(checkpoint_path)


# Define the checkpoint callback
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,  # Save only the model's weights
    verbose=1
)

# Train the model with the checkpoint callback
model.fit(train_images,
          train_labels,
          epochs=12,
          validation_data=(test_images,test_labels),
          callbacks=[checkpoint_callback])


Epoch 1/12
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4845 - loss: 1.6356 
Epoch 1: saving model to training_1/cp.ckpt.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.4957 - loss: 1.6073 - val_accuracy: 0.7880 - val_loss: 0.7020
Epoch 2/12
29/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8688 - loss: 0.4547
Epoch 2: saving model to training_1/cp.ckpt.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.8695 - loss: 0.4520 - val_accuracy: 0.8330 - val_loss: 0.5339
Epoch 3/12
30/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9245 - loss: 0.2797
Epoch 3: saving model to training_1/cp.ckpt.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9235 - loss: 0.2811 - val_accuracy: 0.8450 - val_loss: 0.4873
Epoch 4/12
30/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9365 - loss: 0.2179
Epoch 4: saving model to training_1/cp.ckpt.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9376 - loss: 0.2176 - val_accuracy: 0.857

In [9]:
model = create_model()
loss,acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


32/32 - 0s - 13ms/step - accuracy: 0.1080 - loss: 2.3170
Untrained model, accuracy: 10.80%


In [10]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


32/32 - 0s - 5ms/step - accuracy: 0.8790 - loss: 0.3988
Restored model, accuracy: 87.90%
